## `pyvo.mivot` package demonstrator

The code below shows how the model views provided by the  `pyvo.mivot` package can be used.

- Both code and data are  mostly taken out if the `pyvo.mivot` test suite. 

It aims at showing how the model views provided by the package can be used.

- This examples are based on the initual version of the `pyvo.mivot` package.
- The code is totaly model agnostic
- There is no connection with Astropy classes

The process is 3 steps:
1. The MIVOT block (XML) is read by the viewer and references to table columns and GLOBALS objects are then resolved
2. The XMl block is tranformed in a dictionary where dm roles are sued as keys.
3. This dictionary is used to buikd a dynamic Python object that ca be used as any other object.
    1. The keys od the dictionary are transformed in object attributes.
    2. Dynamic object have no class. Instances can be modified but new instance cannot be created bu by doing copies.

## Watchout

This notebook has been develop before pyvo 1.6 has been published: works with the developement code. Pyvo is supposed to be clone at the same level as `dm-usecases`

In [65]:
%pwd
%cd ../../pyvo
%pwd
!pip install -e .
%cd ../dm-usecases/notebooks
%pwd

/Users/laurentmichel/Documents/seafile/Seafile/workspaces/git/pyvo
Obtaining file:///Users/laurentmichel/Documents/seafile/Seafile/workspaces/git/pyvo
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for pyvo (pyproject.toml) ... done
  Created wheel for pyvo: filename=pyvo-1.4.dev516+g4b28ccb-0.editable-py3-none-any.whl size=5527 sha256=b4cc8af39557fa9fa12c5b53ddb13a0522f0ab336f151c85c4ee240eabee0aab
  Stored in directory: /private/var/folders/14/2k5j91vn0mv2c_k4zdlc4yq00000gp/T/pip-ephem-wheel-cache-i9u5hg08/wheels/90/0b/30/362e901b2a2c78afd50a7a3245317fae0065fc7a8bd85650d0
Successfully built pyvo
  Attempting uninstall: pyvo
    Found existing installation: pyvo 1.4.dev516+g4b28ccb
    Uninstalling pyvo-1.4.dev516+g4b28ccb:
      Successfully uninstalled pyvo-1.4.dev516+g4b28ccb
/Users/laurentmichel/Documen

'/Users/laurentmichel/Documents/seafile/Seafile/workspaces/git/dm-usecases/notebooks'

## Acivate the feature

`pyvo.mivot`is a new pyvo feature, it must be explicetly activated.

In [66]:
from pyvo.mivot import MivotViewer
from pyvo.utils import activate_features

activate_features('MIVOT')

## Connect the MIVOT viewer with the VOTable.

The first data row is read and interpreted and the iterator is rewound.

In [67]:
votable = "../pyvo-ci-sample/gaia_epoch_propagation_full.xml"
# init the MIVOT viewer from a VOTable file, parsed VOTable or DAL response
mivot_viewer = MivotViewer(votable)

## Get an XML serialization of the mapped model

When the viewer is created it reads the first data row to intialize the model view.
1. The MIVOT block (XML) is read and references are resolved
    1. referenced objects are copied at the right places
    2. values of ATTRIBUTE with @ref are set with the values of the matching FIELD.
2. Some XML attributes are added to facilitate further processing (or development)

The XML output, pretty printed here, can be used as such with e.g. XPath queries.

In [68]:
from pyvo.mivot.utils.xml_utils import XmlUtils
# print out the XML serialization of the mapped object
XmlUtils.pretty_print(mivot_viewer.xml_view)

<TEMPLATES>
  <INSTANCE dmtype="mango:EpochPosition">
    <ATTRIBUTE dmrole="mango:EpochPosition.longitude" dmtype="ivoa:RealQuantity" ref="RA_ICRS" unit="deg" col_index="3" field_unit="deg" value="307.79115807079">
      </ATTRIBUTE>
  <ATTRIBUTE dmrole="mango:EpochPosition.latitude" dmtype="ivoa:RealQuantity" ref="DE_ICRS" unit="deg" col_index="4" field_unit="deg" value="20.43108005561">
      </ATTRIBUTE>
  <ATTRIBUTE dmrole="mango:EpochPosition.parallax" dmtype="ivoa:RealQuantity" unit="mas" ref="Plx" col_index="8" field_unit="mas" value="0.4319">
      </ATTRIBUTE>
  <ATTRIBUTE dmrole="mango:EpochPosition.radialVelocity" dmtype="ivoa:RealQuantity" unit="km/s" ref="RV" col_index="25" field_unit="km / s" value="--">
      </ATTRIBUTE>
  <ATTRIBUTE dmrole="mango:EpochPosition.pmLongitude" dmtype="ivoa:RealQuantity" unit="mas/yr" ref="pmRA" col_index="11" field_unit="mas / yr" value="-2.557">
      </ATTRIBUTE>
  <ATTRIBUTE dmrole="mango:EpochPosition.pmLatitude" dmtype="ivoa:RealQuan

## Get a mivot instance

The code below gets the MIVOT instance generated by the viewer. 

- Note that the data table can be iterated by the viewer in the same way as Astropy.
- Each time a new row is reasd, the MIVOT instance is updated with the new values.
- It recommended to work with a reference on the MIVOT instance for performance reasons.

The MIVOT instance has a representation method that provides a pretty serialization of the dictionary that has been used to build the instance.

In [69]:
# get the reference on the dynamic MIVOT instance
mivot_object = mivot_viewer.dm_instance

# iterate over one data row
while mivot_viewer.next():
    # print out the dictionary that has been use to build the instance
    print(mivot_object)
    break


{
  "dmtype": "EpochPosition",
  "longitude": {
    "value": 307.79115807079,
    "unit": "deg"
  },
  "latitude": {
    "value": 20.43108005561,
    "unit": "deg"
  },
  "parallax": {
    "value": 0.4319,
    "unit": "mas"
  },
  "radialVelocity": {
    "value": null,
    "unit": "km/s"
  },
  "pmLongitude": {
    "value": -2.557,
    "unit": "mas/yr"
  },
  "pmLatitude": {
    "value": -5.482,
    "unit": "mas/yr"
  },
  "epoch": {
    "value": "2016.5"
  },
  "pmCosDeltApplied": {
    "value": true
  },
  "EpochPosition_errors": {
    "dmrole": "errors",
    "dmtype": "EpochPositionErrors",
    "EpochPositionErrors_parallax": {
      "dmrole": "parallax",
      "dmtype": "PropertyError1D",
      "sigma": {
        "value": 0.06909999996423721,
        "unit": "mas"
      }
    },
    "EpochPositionErrors_radialVelocity": {
      "dmrole": "radialVelocity",
      "dmtype": "PropertyError1D",
      "sigma": {
        "value": null,
        "unit": "km/s"
      }
    },
    "EpochPosit

## Access model fields through the MIVOT instance

Mapped quantities can be accessed through paths defined by the model.

To be Python-compliant, path elements that contain ":" or "." are escaped with "_".

Each model leaf has 3 attributes:
1. The value
2. The unit as a string, not an Astropy unit yet
3. The identifier of the column the value comes from (if it exists) 

In [70]:
# init the viewer with a "with" statement
with MivotViewer(votable) as mivot_viewer:
    # get the reference on the dynamic MIVOT instance
    mivot_object = mivot_viewer.dm_instance
    
    # the space frame is supposed to be the same for all table rows.
    space_frame = mivot_object.EpochPosition_coordSys.PhysicalCoordSys_frame.spaceRefFrame.value

    # iterate over all data rows
    while mivot_viewer.next():
        # get the position values and units
        ra = mivot_object.longitude.value
        ra_unit = mivot_object.longitude.unit
        dec = mivot_object.latitude.value
        # get the proper motion values and units
        pmra = mivot_object.pmLongitude.value
        pmra_unit = mivot_object.pmLongitude.unit
        pmdec = mivot_object.pmLatitude.value
        # get the epoch
        epoch =  mivot_object.epoch.value
        # Print out a summary of the row
        print(f"Year {epoch}: position({space_frame}) = [{ra} {dec} {ra_unit}] proper motion = [{pmra} {pmdec} {pmra_unit}]")


Year 2016.5: position(ICRS) = [307.79115807079 20.43108005561 deg] proper motion = [-2.557 -5.482 mas/yr]
Year 2016.5: position(ICRS) = [307.79582391363 20.43253083107 deg] proper motion = [-2.78 -3.853 mas/yr]
Year 2016.5: position(ICRS) = [307.79737366047 20.43558827154 deg] proper motion = [None None mas/yr]
Year 2016.5: position(ICRS) = [307.78856137441 20.43011713943 deg] proper motion = [-2.781 -3.61 mas/yr]
Year 2016.5: position(ICRS) = [307.78977228741 20.43150439876 deg] proper motion = [-1.975 -3.712 mas/yr]


## Run the viewer from a DAL service

The viewer can be directly initialized from a DAL service response.

In the cell below, the query has been tuned to return one row.

In [71]:
import astropy.units as u
from astropy.coordinates import SkyCoord
from pyvo.dal.scs import SCSService

# initialize the DAL service
vizier_url = "https://cdsarc.cds.unistra.fr/beta/viz-bin/mivotconesearch/I/239/hip_main"
scs_srv = SCSService(vizier_url)

# init the viewer from a DAL response.
mivot_viewer = MivotViewer(
    scs_srv.search(
        pos=SkyCoord(ra=52.26708 * u.degree, dec=59.94027 * u.degree, frame="icrs"),
        radius=0.05,
    )
)

# get the reference on the MIVOT instance
mivot_object = mivot_viewer.dm_instance
while mivot_viewer.next():
    # get the space frame reference (could be done out of the loop)
    frame = mivot_object.Coordinate_coordSys.spaceRefFrame.value
    # get the position values and units
    ra = mivot_object.longitude.value
    ra_unit = mivot_object.longitude.unit
    dec = mivot_object.latitude.value
    # get the proper motion values and units
    pmra = mivot_object.pmLongitude.value
    pmra_unit = mivot_object.pmLongitude.unit
    pmdec = mivot_object.pmLatitude.value
    # get the epoch
    epoch =  mivot_object.epoch.value
    # Print out a summary of the row
    print(f"Year {epoch}: position({frame})=[{ra} {dec} {ra_unit}] proper motion = [{pmra} {pmdec} {pmra_unit}]")

Year 1991.25: position(ICRS)=[52.26722684 59.94033461 deg] proper motion = [-0.82 -1.85 mas/yr]


### Calculate the epoch propagation of thye selected object

We compute the `SkyCoord`at the epoch given by Vizier and then we apply the space motion for the 4 following decades.

This functionality will add to the MIVOT package in a next PR.



In [72]:
from astropy.coordinates import SkyCoord  
from astropy.time import Time  
try:
    from erfa import ErfaWarning
except Exception:
    from astropy.utils.exceptions import ErfaWarning
    
position = SkyCoord(ra*u.deg, dec*u.deg,
                    frame=frame.lower(),
                    pm_ra_cosdec=pmra*u.mas/u.year, pm_dec=pmdec*u.mas/u.year, 
                    obstime=Time(epoch, format='decimalyear', scale='utc'))
dt = 10. * u.year

for cpt in range(5):
    print(f"year {position.obstime} {position.to_string(style='hmsdms', sep=':', precision=6)}")
    position = position.apply_space_motion(dt=dt)

year 1991.25 03:29:04.134442 +59:56:25.204596
year 2001.2486299467275 03:29:04.133350 +59:56:25.186096
year 2011.2499997463217 03:29:04.132259 +59:56:25.167596
year 2021.2486297881785 03:29:04.131168 +59:56:25.149096
year 2031.2499996511922 03:29:04.130076 +59:56:25.130596


## Apply the model view on data rows read with Astropy

Once the mivot viewe has been initialized in a regular way, the VOTable data can be iterated with Astropy and the MIVOT viewer can be invoked to provide a model view on any record. 

In [73]:
from astropy.io.votable import parse

# Parse the VOTable with Astropy
parsed_votable = parse(votable)
table = parsed_votable.resources[0].tables[0]

# init the viewer from the parsed VOtable.
mivot_viewer = MivotViewer(votable)

# get the reference on the MIVOT instance
mivot_instance = mivot_viewer.dm_instance

# the space frame is supposed to be the same for all table rows.
space_frame = mivot_instance.EpochPosition_coordSys.PhysicalCoordSys_frame.spaceRefFrame.value

# Iterate over all data rows
for rec in table.array:
    # update the instance with the new data row
    mivot_instance.update(rec)
    # get the position values and units
    ra = mivot_instance.longitude.value
    ra_unit = mivot_instance.longitude.unit
    dec = mivot_instance.latitude.value
    # get the proper motion values and units
    pmra = mivot_instance.pmLongitude.value
    pmra_unit = mivot_instance.pmLongitude.unit
    pmdec = mivot_instance.pmLatitude.value
    # get the epoch
    epoch =  mivot_instance.epoch.value
    # Print out a summary of the row
    print(f"Year {epoch}: position({space_frame}) = [{ra} {dec} {ra_unit}] proper motion = [{pmra} {pmdec} {pmra_unit}]")


Year 2016.5: position(ICRS) = [307.79115807079 20.43108005561 deg] proper motion = [-2.557 -5.482 mas/yr]
Year 2016.5: position(ICRS) = [307.79582391363 20.43253083107 deg] proper motion = [-2.78 -3.853 mas/yr]
Year 2016.5: position(ICRS) = [307.79737366047 20.43558827154 deg] proper motion = [None None mas/yr]
Year 2016.5: position(ICRS) = [307.78856137441 20.43011713943 deg] proper motion = [-2.781 -3.61 mas/yr]
Year 2016.5: position(ICRS) = [307.78977228741 20.43150439876 deg] proper motion = [-1.975 -3.712 mas/yr]
